In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

import os
import gzip
import zipfile
import pandas as pd
import numpy as np
import scipy.io.wavfile as wav
import csv
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [22]:

data = pd.DataFrame(columns=['label', 'feature'])
#data_dir = "/content/gdrive/My Drive/TrainTmp/Train/"
data_dir = "/content/gdrive/My Drive/Handout (1)/Train/Train/"
_len = 0
input_duration = 3
maxx = 0
with open('/content/gdrive/My Drive/Handout (1)/train_label.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)    # bỏ qua dòng đầu tiên của file csv
    for row in csv_reader:
      _len += 1
      
      X, sample_rate = librosa.load(data_dir + row[0], res_type='kaiser_fast',duration=input_duration,sr=22050*2,offset=0.5)
      #sample_rate, X = wav.read(data_dir + row[0])
      #sample_rate = np.array(sample_rate)
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=16000, n_mfcc=13), axis=0)
      feature = mfccs
      data.loc[_len, 'feature'] = feature
      data.loc[_len, 'label'] = int(row[1])
      maxx  = max(maxx, feature.shape[0])
      print(_len)

    

In [0]:

for i in range(1, _len + 1):
  Tmp = np.zeros(maxx)
  dat  = data['feature'].loc[i]

  for j in range(dat.shape[0]):
    Tmp[j] = dat[j]
  data['feature'].loc[i] = Tmp


In [0]:
X = pd.DataFrame(data['feature'].values.tolist())
#X = np.array(data['feature'])
#Y = np.array(data['label'])
X = np.array(X)
Y = pd.DataFrame(data['label'].values.tolist())
Y = np.array(Y)
Y = np.resize(Y, _len)
X = np.resize(X, (_len, maxx))

In [0]:
def plot_time_series(data):
    """
    Plot the Audio Frequency.
    """
    fig = plt.figure(figsize=(14, 8))
    plt.title('Raw wave ')
    plt.ylabel('Amplitude')
    plt.plot(np.linspace(0, 1, len(data)), data)
    plt.show()


def noise(data):
    """
    Adding White Noise.
    """
    # you can take any distribution from https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html
    noise_amp = 0.005*np.random.uniform()*np.amax(data)
    data = data.astype('float64') + noise_amp * np.random.normal(size=data.shape[0])
    return data
    
def shift(data):
    """
    Random Shifting.
    """
    s_range = int(np.random.uniform(low=-5, high = 5)*500)
    return np.roll(data, s_range)
    
def stretch(data, rate=0.8):
    """
    Streching the Sound.
    """
    data = librosa.effects.time_stretch(data, rate)
    return data
    
def pitch(data, sample_rate):
    """
    Pitch Tuning.
    """
    bins_per_octave = 12
    pitch_pm = 2
    pitch_change =  pitch_pm * 2*(np.random.uniform())   
    data = librosa.effects.pitch_shift(data.astype('float64'), 
                                      sample_rate, n_steps=pitch_change, 
                                      bins_per_octave=bins_per_octave)
    return data
    
def dyn_change(data):
    """
    Random Value Change.
    """
    dyn_change = np.random.uniform(low=1.5,high=3)
    return (data * dyn_change)
    
def speedNpitch(data):
    """
    peed and Pitch Tuning.
    """
    # you can change low and high here
    length_change = np.random.uniform(low=0.8, high = 1)
    speed_fac = 1.0  / length_change
    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)
    minlen = min(data.shape[0], tmp.shape[0])
    data *= 0
    data[0:minlen] = tmp[0:minlen]
    return data


array([0.00219577, 0.00204032, 0.00182946, ..., 0.00611031, 0.00559541,
       0.00500626], dtype=float32)

In [41]:
#X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state = 42, shuffle = True)

#clf = SVC(kernel = 'rbf', probability=True)


#clf.fit(X_train, y_train)

#print(accuracy_score(clf.predict(X_val), y_val))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.27724665391969405


In [42]:
# Define the paramter grid for C from 0.001 to 10, gamma from 0.001 to 10
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state = 42, shuffle = True)

C_grid = [0.001, 0.01, 0.1, 1, 10]
gamma_grid = [0.001, 0.01, 0.1, 1, 10]
param_grid = {'C': C_grid, 'gamma' : gamma_grid}

grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv = 3, scoring = "accuracy")
grid.fit(X_train, y_train)

# Find the best model
print(grid.best_score_)

print(grid.best_params_)

print(grid.best_estimator_)

0.3452067893856084
{'C': 1, 'gamma': 0.001}
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [43]:
clf = SVC(kernel = 'rbf', C = 1, gamma = 0.001, probability=True)

clf.fit(X_train, y_train)

print(accuracy_score(clf.predict(X_val), y_val))

0.36233269598470363


In [2]:
%cd '/content/gdrive/My Drive/Handout (1)'
! ls

/content/gdrive/My Drive/Handout (1)
Train  train_label.csv


In [0]:
! pip install os


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
